In [1]:
from pathlib import Path
from tmc_summarizer.data_model import TMC_File
tmc_path = '/mnt/g/My Drive/TMCs/1_Central_Ave.xls'

In [2]:
tmc = TMC_File(tmc_path)
# tmc.df_cars.head(50)



Reading 1_Central_Ave.xls
